Codigo de prueba Spark

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("GoogleColabSpark").getOrCreate()
print(spark.version)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-12-03 17:49:35,024 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


3.2.1


Se importa librerias necesarias

In [32]:
import os
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max, desc

Inicialización de Spark

In [33]:
spark=SparkSession.builder.master("local[*]").appName("ProcesamientoCrimenEspaña").getOrCreate()

Se verifica la ruta adecuada

In [34]:
import os
print(os.getcwd())

/home/221c1129yulbrynner/Procesamiento_Spark


In [26]:
#import os
#os.chdir('/home/221c1129yulbrynner/Procesamiento_Spark')

Configuración para que use solo local

In [35]:
spark.conf.set("fs.defaultFS", "file:///")

Carga del dataset

In [36]:
ruta_csv="Tasa_de_crimen_España_2019_2021.csv"
datos_crimenes=spark.read.csv(ruta_csv, header=True, inferSchema=True)

Visualización del esquema de datos

In [37]:
datos_crimenes.printSchema()

root
 |-- Location: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Crime: string (nullable = true)
 |-- Total cases: integer (nullable = true)



Tasa de crimen promedio por año

In [38]:
tasa_promedio_anual = datos_crimenes.groupBy("Year").agg(avg(col("Total cases")).alias("Tasa_promedio"))
tasa_promedio_anual.show()

+----+-----------------+
|Year|    Tasa_promedio|
+----+-----------------+
|2019|6562.745535714285|
|2020|4991.915178571428|
|2021|5803.651785714285|
+----+-----------------+



Ciudad con mayor tasa de crimen en cada año

In [39]:
ciudad_maxim_crimen = datos_crimenes.groupBy("Year", "Location").agg(max(col("Total cases")).alias("Tasa_maxima"))
ciudad_maxim_crimen.orderBy(desc("Tasa_maxima")).show()

+----+---------+-----------+
|Year| Location|Tasa_maxima|
+----+---------+-----------+
|2019|   Madrid|     205312|
|2021|   Madrid|     197970|
|2020|   Madrid|     181917|
|2019|Barcelona|     165901|
|2021|Barcelona|     152565|
|2020|Barcelona|     135216|
|2021| Valencia|      74380|
|2019| Valencia|      68853|
|2020| Valencia|      64822|
|2021| Alicante|      49211|
|2021|  Seville|      48461|
|2019|  Seville|      48365|
|2019| Alicante|      48178|
|2021|   Malaga|      47803|
|2020|  Seville|      46704|
|2020| Alicante|      45607|
|2019|   Malaga|      44516|
|2020|   Malaga|      42372|
|2021|   Murcia|      37088|
|2020|   Murcia|      32819|
+----+---------+-----------+
only showing top 20 rows



Tipo de crimen con tasa más baja

In [40]:
tipo_crimen_minim=datos_crimenes.groupBy("Crime").agg(avg(col("Total cases")).alias("Tasa_promedio"))
tipo_crimen_minim.orderBy("Tasa_promedio").show()

+--------------------+------------------+
|               Crime|     Tasa_promedio|
+--------------------+------------------+
|          Kidnapping|            3.1875|
|Intentional homic...|11.291666666666666|
|Intentional homic...|             34.25|
|Sexual assault wi...| 70.70833333333333|
|Other crimes agai...|           495.625|
|Crimes against fr...| 566.3541666666666|
|Serious and less ...| 663.4166666666666|
|    Drug trafficking| 681.3958333333334|
|       Vehicle theft|1345.6041666666667|
|Robbery with viol...|2801.8541666666665|
|Robberies with fo...|3143.9583333333335|
|Burglaries with f...|3535.2083333333335|
|               Theft|23808.645833333332|
|Other criminal of...|43843.958333333336|
+--------------------+------------------+



Guardar resultados procesados

Se guarda datos de la tasa promedio actual

In [42]:
ruta_salida_tasa_promedio = "tasa_promedio_anual.csv"
tasa_promedio_anual.coalesce(1).write.csv(ruta_salida_tasa_promedio, header=True, mode="overwrite")

In [45]:
ruta_salida_ciudad_max = "ciudad_maximo_crimen.csv"
ciudad_maxim_crimen.coalesce(1).write.csv(ruta_salida_ciudad_max, header=True, mode="overwrite")

In [44]:
ruta_salida_tipo_crimen_min = "tipo_crimen_minimo.csv"
tipo_crimen_minim.coalesce(1).write.csv(ruta_salida_tipo_crimen_min, header=True, mode="overwrite")